# script for processing rolling cv

# Runs the file containing general functions used in this script
## By default this is a file called "methane_functions.r" in the working directory of this script
## Do NOT change the working directory at any point while using this script.

In [ ]:
methaneFunctions<-paste(getwd(),"/methane_functions.r",sep="")
methaneFunctions
if (file.exists(methaneFunctions)){
    source(methaneFunctions)
} else {
    print("manually correct the location and/or name of methane_functions.r")
}

# 0. Load packages used

In [ ]:
# # Package names
# packages<-c("dplyr", "lubridate", "ggplot2", "ggpubr")
# packageHandler(packages)

## Function to calculate rolling CV for warm-up detection
Starting with using the summary data for calculations

In [ ]:
calculateCV<-function(processed_data, dataType, lc=loggerCount, ll=loggerList){
    temp_cv<-data.frame()
    calculated_cv_list<-vector("list", lc)
    for(i in 1:lc){
        df<-subset(processed_data, logger==ll[i] & type==dataType)

        measurementCycles<-unique(df$measurementCycle)
        for(i in 1:length(measurementCycles)){
            temp<-subset(df,measurementCycle==measurementCycles[i])
            temp$roll_mean<-rollapply(data=temp$ch4_raw,width=4,align=c("right"),FUN=mean,fill=NA)
            temp$roll_sd<-rollapply(data=temp$ch4_raw,width=4,align=c("right"),FUN=sd,fill=NA)
            temp$roll_cv<-temp$roll_sd/temp$roll_mean #optionally multiply by 100 for %
            temp_cv<-bind_rows(temp_cv,temp)
        }
        calculated_cv_list[[i]]<-temp_cv
    }
    return(bind_rows(calculated_cv_list))
}

## Function to create basic plots of burst vs raw methane reading colored by measurement cycle

In [ ]:
cycleVsCH4_plots<-function(processed_data, dataType, lc=loggerCount, ll=loggerList){
    plots<-vector("list", lc)
    names(plots)<-ll
    for(i in 1:lc){
        data<-subset(processed_data, logger==ll[i] & type==dataType)

        plots[[i]]<-ggplot(data, aes(burstCycle, ch4_raw))+
            geom_point(aes(color=measurementCycle))+scale_color_viridis()+
            ggtitle(paste("Logger: ", ll[i], "\nData type: ", dataType))
    }
    return(plots)
}

## Function to plot rolling cv vs burst cycle and colored by measurement cycle

In [ ]:
rollCV_plots<-function(parseDataCV, ll=loggerList, lc=loggerCount){
    plots<-vector("list", lc)
    names(plots)<-ll
    for(i in 1:lc){
        data<-subset(parseDataCV, logger==ll[i])
        plots[[i]]<-ggplot(data, aes(burstCycle, roll_cv))+
        geom_point(aes(color=as.integer(measurementCycle)))+
        scale_y_log10()+scale_color_viridis()+
        ggtitle(paste("Rolling CV for Logger: ",ll[i]))
    }
    return(plots)
}

## Function to calculate the mean of rolling cv above provided burst for each measurement cycle to determine target cv for methane driver

In [ ]:
mean_cv_burst<-function(parseDataCV, burstCycleStart, ll=loggerList, lc=loggerCount){
    mean_na<-function(x){mean(x,na.rm=TRUE)}
    
    temp_mean<-vector("list", lc)
    for(i in 1:lc){
        df<-subset(parseDataCV, logger==ll[i] & as.integer(burstCycle)>=burstCycleStart)
        
        if(max(df$measurementCycle) > 2){
            temp_mean[[i]]<-summaryBy(roll_cv~measurementCycle,df,FUN=c(mean_na))
            temp_mean[[i]]$logger<-unique(df$logger)
            temp_mean[[i]]$deployed_at<-lubridate::as_datetime(unique(df$deployed_at))
        }
    }
    return(bind_rows(temp_mean))
}

# RUN

# TODO: need the lines for initial concatenation of data in relevant directory

### Create plots of raw data and summary data and save them

In [ ]:
raw_cycleVsCH4_plots<-cycleVsCH4_plots(processed_data, "raw")
summary_cycleVsCH4_plots<-cycleVsCH4_plots(processed_data, "summary")

savePlotList(raw_cycleVsCH4_plots, "raw_cycleVsCH4_")
savePlotList(summary_cycleVsCH4_plots, "summary_cycleVsCH4_")

## calculate the mean cv of each burst and save it

In [ ]:
meanCVburst<-mean_cv_burst(parseSummaryDataCV, 20)

saveDFrds(meanCVburst)
saveDFcsv(meanCVburst)

### Print results

In [ ]:
for(i in 1:length(loggerList)){
    print(loggerList[i])
    print(subset(meanCVburst, logger==loggerList[i]))
}

### Use this ^ data to decide a CV limit indicating when sensor has been sufficiently warmed
### Function to go through each logger and each measurement cycle and save the first instance where CV limit has been reached to an output dataframe

In [ ]:
# first instance of CV lower than 0.01 for each measurementCycle indicates a good reading
# readings before that are not warmed up, readings after that are good but consume too much power
# Ideally we want 1 value (1 summary line and/or 10 raw lines from burst) per measurement cycle

# for each logger, for each measurement cycle, save 1 line to new dataframe
manualCVlimit<-function(parseSummaryDataCV, manualCV, ll=loggerList, lc=loggerCount){
    # TODO: first row is populated with NAs
    outputDF<-data.frame(matrix(ncol=length(parseSummaryDataCV), nrow=0))
    colnames(outputDF)<-names(parseSummaryDataCV)

    for(i in 1:lc){
        oneLogger<-subset(parseSummaryDataCV, logger==ll[i])

        measurementCycles<-length(unique(oneLogger$measurementCycle))

        for(j in 1:measurementCycles){
            oneCycle<-subset(oneLogger, measurementCycle==j)
            
            #save the first instance where the rolling cv has reached the desired limit
            # TODO: save line with the lowest CV reached if has not reached the desired limit
            # errors not an issue currently: reporting warnings where CV not calculated or mostly NA values
            rowFound<-min(which(oneCycle$roll_cv<manualCV), na.rm=TRUE)
            if(!is.integer(rowFound)){
                print(sprintf("cv limit not reached for logger:%s measurementCycle:%d", ll[i], j))
                print(sprintf("lowest cv is: %f", min(oneCycle$roll_cv, na.rm=TRUE)))
            }else{
                outputDF<-rbind(outputDF,oneCycle[rowFound,])
            }
        }
    }
    outputDF<-process_columns(outputDF)
    # either sort the categorical factor burst cycles or change them to numeric here:
    outputDF$burstCycle<-as.numeric(outputDF$burstCycle)
    return(outputDF)
}

### Subset good data and save it to rds and csv

In [ ]:
summary_manualCVdata<-manualCVlimit(parseSummaryDataCV, 0.01)
# outputDF$measurementCycle<-as.numeric(outputDF$measurementCycle)

saveDFrds(summary_manualCVdata)
saveDFcsv(summary_manualCVdata)

#TODO: mean burstCycle(how many minutes it took to warm up) & standard error, and actual cv value

In [ ]:
loggerPlots<-plot_Data_v_Time(summary_manualCVdata, "site")
loggerPlots

savePlotList(loggerPlots, "sp_manualCV_")

In [ ]:
individualPlots<-plot_individual_logger_data_v_time(summary_manualCVdata)

individualPlots

savePlotListList(individualPlots, "ip_manualCV_")